# Task 2 (50 marks) Application of Knowledge Graph

neo4j.bat console

In [1]:
from py2neo import Graph, Subgraph, Node, Relationship
import pandas as pd
from neo4j import GraphDatabase

In [14]:
user = 'neo4j'
password = 'X2d00y1!'
uri= 'bolt://localhost:7687'
driver = GraphDatabase.driver(uri, auth=(user, password))
try:
    graph = Graph("http://localhost:7474", auth=(user, password))
    print("Successfully connected to Neo4j!")
except Exception as e:
    print(f"Error connecting to Neo4j: {e}")

Successfully connected to Neo4j!


In [15]:
session = driver.session()

result = session.run("MATCH (n) DETACH DELETE n")

print("图已成功清空。")

图已成功清空。


## Question 3: (10 marks) Constructing a Knowledge Graph

In [16]:
def batch_create(graph, nodes_list, relations_list):
    subgraph = Subgraph(nodes_list, relations_list)
    tx_ = graph.begin()
    tx_.create(subgraph)
    graph.commit(tx_)

In [ ]:
nodes_list = []
with open(r"C:\Users\CANDY\Desktop\to do\作业\dsaa5002_project\KnowledgeGraph\hidy.nodes.company.csv", "r",encoding="utf-8") as node_file:
    next(node_file)  # 跳过标题行
    for line in node_file:
        node_data = line.strip().split(",")
        node_id, company_name, code, label = node_data
        print(node_id,company_name)
        nodes_list.append(Node("Company", id=int(node_id), name=company_name, code=code))

#len(nodes_list)##3974
batch_create(graph, nodes_list, [])

In [19]:
def relation(name):
    relationlist=[]
    path = r"C:\Users\CANDY\Desktop\to do\作业\dsaa5002_project\KnowledgeGraph\hidy.relationships.{}.csv".format(name)
    with open(path, "r") as relationship_file:
        next(relationship_file)  # 跳过标题行
        for line in relationship_file:
            relationship_data = line.strip().split(",")[:3]
            #start_id, end_id, relation_type = relationship_data
            start_id, end_id = relationship_data
            start_node = graph.nodes.match(id=int(start_id)).first()
            end_node = graph.nodes.match(id=int(end_id)).first()
            
            relationlist.append(Relationship(start_node, name,end_node))
    batch_create(graph, [], relationlist)
    print(name,len(relationlist))

In [20]:
relation("compete")
relation("cooperate")
relation("dispute")
relation("invest")
relation("same_industry")
relation("supply")

compete 25
cooperate 3603
dispute 439
invest 559
same_industry 5596
supply 1444


In [1]:
driver.close()

NameError: name 'driver' is not defined

## Question 4 (20 marks): Knowledge-Driven Financial Analysis

In [94]:
import pandas as pd

# 读取数据
task1 = pd.read_excel(r"C:\Users\CANDY\Desktop\to do\作业\dsaa5002_project\Task1.xlsx")
company_df = pd.read_csv(r"C:\Users\CANDY\Desktop\to do\作业\dsaa5002_project\KnowledgeGraph\hidy.nodes.company.csv", encoding="utf-8")
relations = ['compete', 'cooperate', 'dispute', 'invest', 'same_industry', 'supply']
relation_dfs = [pd.read_csv(r'C:\Users\CANDY\Desktop\to do\作业\dsaa5002_project\KnowledgeGraph\hidy.relationships.{}.csv'.format(rel), encoding="utf-8") for rel in relations]


In [ ]:
result_columns = ['NewsID', 'NewsContent', 'Explicit_Company', 'label', 'Implicit_Positive_Company',
                  'Implicit_Negative_Company']

In [177]:
def process_relation(row, relation_df, company_df):
    relation_type = relation_df[':TYPE'].unique()[0]  # 获取关系类型
    explicit_companies = row['Explicit_Company'].split(',')  # 将多个公司分隔成列表
    matched_ids = []

    # 在company_df中查找与explicit_companies匹配的ID
    for company in explicit_companies:
        matched_ids.extend(company_df.loc[company_df['company_name'] == company, ':ID'].tolist())

    # 在relation_df中过滤与匹配的ID相关的关系
    related_ids = relation_df.loc[relation_df[':START_ID'].isin(matched_ids), ':END_ID'].tolist()
    related_ids.extend(relation_df.loc[relation_df[':END_ID'].isin(matched_ids), ':START_ID'].tolist())
    related_ids = list(set(related_ids))

    # 判断关系类型并处理对应的公司
    related_companies = company_df.loc[company_df[':ID'].isin(related_ids), 'company_name'].tolist()
    if row['label'] == 1:
        if relation_type in ['same_industry', 'cooperate', 'invest', 'supply']:
            row['Implicit_Positive_Company'].extend(related_companies)
        elif relation_type in ['compete', 'dispute']:
            row['Implicit_Negative_Company'].extend(related_companies)
    elif row['label'] == 0:
        if relation_type in ['same_industry', 'cooperate', 'invest', 'supply']:
            row['Implicit_Negative_Company'].extend(related_companies)
        elif relation_type in ['compete', 'dispute']:
            row['Implicit_Positive_Company'].extend(related_companies)
    return row

In [181]:
def process_row(row):
    implicit_positive_companies = []
    implicit_negative_companies = []

    for relation_df in relation_dfs:
        row['Implicit_Positive_Company'] = implicit_positive_companies
        row['Implicit_Negative_Company'] = implicit_negative_companies
        row = process_relation(row, relation_df, company_df)

    implicit_positive_companies = list(set(row['Implicit_Positive_Company']))
    implicit_negative_companies = list(set(row['Implicit_Negative_Company']))

    result_row = pd.Series([row['NewsID'], row['NewsContent'], row['Explicit_Company'], row['label'],
                            ','.join(implicit_positive_companies) if implicit_positive_companies else None,
                            ','.join(implicit_negative_companies) if implicit_negative_companies else None],
                          index=result_columns)

    return result_row

In [ ]:
# 使用apply方法处理每一行
result = task1.apply(lambda row: process_row(row), axis=1)

# 将结果数据框的列名改为result_columns

result.columns = result_columns

In [182]:
result

,NewsID,NewsContent,Explicit_Company,label,Implicit_Positive_Company,Implicit_Negative_Company
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，北京市第...,建设银行,0,"捷捷微电,任子行","交通银行,佳都科技,中国石化,我爱我家,晨鸣纸业,贵州茅台,中信银行,南天信息,特发信息,中..."
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上表示，此...,农业银行,1,"交通银行,神州信息,润建股份,牧原股份,紫金银行,南天信息,中信银行,东方财富,优博讯,神州...",ST云维
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权...,中国国航,1,"中国电建,南方航空,兴业证券,寒武纪,中国石化,吉祥航空,山航B,春秋航空,中国卫通,农业银...",None
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元...,胜利股份,1,"新疆浩源,特锐德",None
4,8,由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大减，同时满洲里口岸铁路在修复线，导致中...,"冠农股份,雅化集团",1,"富邦股份,东华科技,天齐锂业,立讯精密,南宁百货,永兴材料,宁德时代,西部黄金,藏格控股,德...",None
...,...,...,...,...,...,...
396376,1037031,每经AI快讯，有投资者在投资者互动平台提问：请问公司目前有没有电解槽产能，规划情况能否详细介...,亿华通,1,"福田汽车,东风汽车,仕佳光子,飞龙股份,百奥泰,宝泰隆,中国船舶,东旭光电",None
396377,1037032,依米康（SZ 300249，收盘价：10.38元）发布公告称，2023年10月12日，依米康...,中泰证券,1,"贵州茅台,乐歌股份,光大证券,璞泰来,涪陵榨菜,中金公司,翔丰华,长江证券,东方证券,金融街...","西水股份,华谊嘉信"
396378,1037033,天风证券10月13日发布研报称，给予中核科技（000777.SZ，最新价：13.03元）买入...,"中核科技,天风证券",1,"国信证券,人福医药,复旦微电,久远银海,中科软,中金公司,长江证券,西南证券,建龙微纳,华泰...","吉翔股份,三特索道,中源家居"
396379,1037034,有投资者提问：抗癌药CPT获批后，公司是否应该按照股权协议继续收购沙东股权，适应症为MM的C...,海特生物,1,"海尔生物,药明康德",None


In [183]:
result.to_excel(r"C:\Users\CANDY\Desktop\to do\作业\dsaa5002_project\Task2.xlsx", index=False)

In [184]:
import pandas as pd

# 假设您的DataFrame名为result

# 将"None"值替换为"None"字符串
result_filled = result.fillna("None")

# 保存DataFrame为Excel文件，并指定index=False以避免保存索引列
result_filled.to_excel(r"C:\Users\CANDY\Desktop\to do\作业\dsaa5002_project\Task3.xlsx", index=False)